In [1]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp



In [66]:
import numpy as np
import json
import pickle
%config Completer.use_jedi = False



In [167]:
with open(ppj("OUT_DATA", "super_star_avg_prices_2_agents.pickle"), "rb") as f:
    super_star_avg_prices_2_agents = pickle.load(f)

In [168]:
super_star_avg_prices_2_agents.shape

(1000,)

In [169]:
with open(ppj("OUT_DATA", "data_group_level.pickle"), "rb") as f:
    data_group_level = pickle.load(f)

In [170]:
data_group_level

,round,super_game,r_sg,group_id_general,treatment,winning_price,collusive,super_group_id_general,price_algorithm
0,1.0,1.0,0,session_20052021_1000_1,2H0A,3.0,0,session_20052021_1000_1,NaN
2,1.0,1.0,0,session_20052021_1000_2,2H0A,1.0,0,session_20052021_1000_1,NaN
4,1.0,1.0,0,session_20052021_1000_3,2H0A,2.0,1,session_20052021_1000_1,NaN
6,1.0,1.0,0,session_20052021_1000_4,2H0A,1.0,0,session_20052021_1000_2,NaN
8,1.0,1.0,0,session_20052021_1000_5,2H0A,3.0,0,session_20052021_1000_2,NaN
...,...,...,...,...,...,...,...,...,...
460,10.0,3.0,51,session_08062021_1630_3H0A_2,3H0A,4.0,1,session_08062021_1630_3H0A_1,NaN
461,10.0,3.0,51,session_08062021_1630_3H0A_3,3H0A,4.0,1,session_08062021_1630_3H0A_1,NaN
468,11.0,3.0,52,session_08062021_1630_3H0A_1,3H0A,4.0,1,session_08062021_1630_3H0A_1,NaN
469,11.0,3.0,52,session_08062021_1630_3H0A_2,3H0A,4.0,1,session_08062021_1630_3H0A_1,NaN


In [171]:
data_last_super_game = data_group_level.loc[data_group_level['super_game'] == 3].groupby(['super_group_id_general', 'treatment'], as_index=False)[['winning_price']].mean()

In [172]:
from scipy.stats import mannwhitneyu

In [176]:
mannwhitneyu(
    data_last_super_game.loc[data_last_super_game['treatment'] == '2H0A']['winning_price'],
    super_star_avg_prices_2_agents
)

MannwhitneyuResult(statistic=610.5, pvalue=1.238724771685483e-19)